# SQL: Connecting to Data

Agenda:
1. Introduction to Python sql: `sqlite3` module
2. Connecting to SQL Server or other with ODBC, such as (see https://github.com/mkleehammer/pyodbc/wiki):
	* Access
	* Excel
	* MySQL
	* Netezza
	* Oracle
	* Postgres
	* Teradata
	* Vertica
3. Using an ORM (object relational mapping)

## sqlite3

Sqlite is a 'cheap', mostly compliant SQL server which is useful in embedded applications since it doesn't require a server. If you wanted to ship your Project1 and wanted to use a database, you would include sqlite built-in.

The Python standard library comes with the `sqlite3` module. We are using this simplified library because:
1. It provides a simple introduction, without external modules
2. It, along with most Python database libraries, uses the same API (i.e., the same function calls).
	* https://www.python.org/dev/peps/pep-0249/

The basic API pattern is:
1. Make a connection using a connection string
2. Get a cursor to interact with the connection
3. Use the cursor to execute SQL statements
4. Commit all in that connection
5. Close the connection

In [2]:
import sqlite3 
conn = sqlite3.connect('example.db')  # step 1
cur = conn.cursor()  # step 2
cur.execute('''
  CREATE TABLE project (
    id integer primary key,
    name text not null,
    begin_date date,
    end_date date
  ) 
''')  # step 3
cur.execute('''
  INSERT INTO project (name, begin_date, end_date)
  VALUES ('taper', '2018-03-01', '2019-01-01')
''')  # step 3b
conn.commit()  # step 4
conn.close()  # step 5

I'm using PyCharm, which allows viewing the database we've just created. If you don't have a tool, try something like "DB Browser for Sqlite": 
* https://github.com/sqlitebrowser/sqlitebrowser/releases

In [3]:
# we would like to parametrize these values (not hard-code them), especially 
#   if we're building this into an application
conn = sqlite3.connect('example.db')
cur = conn.cursor()
project = 'cannabis'
begin_date = '2018-02-02'
end_date = '2018-12-31'
cur.execute('''
  INSERT INTO project (name, begin_date, end_date)
  VALUES ('{}', '{}', '{}')
'''.format(project, begin_date, end_date))
conn.commit()

In [4]:
# let's make this into a function
def insert(conn, project, begin_date, end_date):
    cur = conn.cursor()
    cur.execute('''
      INSERT INTO project (name, begin_date, end_date)
      VALUES ('{}', '{}', '{}')
    '''.format(project, begin_date, end_date))

What's wrong with this? It makes the database open to a SQL injection attack. https://xkcd.com/327/

Instead, we need to 'sanitize' our input by making sure that nothing bad is included. All SQL engines give us the opportunity to have this done automatically using a `?`.

In [8]:
# let's make this into a function
def insert(conn, project, begin_date, end_date):
    cur = conn.cursor()
    cur.execute('''
      INSERT INTO project (name, begin_date, end_date)
      VALUES (?, ?, ?)
    ''', (project, begin_date, end_date))
    return cur.lastrowid  # return project id

In [9]:
insert(conn, '1b', '2018-01-20', '2019-01-01')
conn.commit()

In [10]:
# querying data
cur = conn.cursor()
cur.execute('select * from project where name = "1b"')

In [11]:
# get the rows by using a for-loop on a cursor
for row in cur:
    print(row)

(3, '1b', '2018-01-20', '2019-01-01')
(4, '1b', '2018-01-20', '2019-01-01')


In [12]:
def get_name(conn, project_name):
    cur = conn.cursor()
    cur.execute('select * from project where name = ?', (project_name, ))
    return [row for row in cur]

In [13]:
get_name(conn, '1b')

[(3, '1b', '2018-01-20', '2019-01-01'), (4, '1b', '2018-01-20', '2019-01-01')]

In [49]:
cur.execute('''
  CREATE TABLE task (
    id integer primary key,
    name text not null,
    priority integer,
    status_id integer,
    project_id integer,
    due_date date
  ) 
''')
conn.cursor()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ProgrammingError: The cursor's connection has been closed.

## pyodbc

Pyodbc is included in the Anaconda distribution (but is not a default library).

The tricky bit is figuring out the connection string. You can get some help searching on Stackoverflow or in the documentation here: https://github.com/mkleehammer/pyodbc/wiki.

In [14]:
import pyodbc

In [16]:
# for sql server (you will need a sql server installation)
conn = pyodbc.connect(driver='SQL Server', server='test', database='test')
conn = pyodbc.connect(
    'DRIVER={SQL Server};SERVER=test;DATABASE=test;Trusted_Connection=yes;'
)

In [17]:
cur = conn.cursor()
cur.execute('''
  CREATE TABLE project (
    id integer primary key identity(1, 1),
    name varchar(50) not null,
    begin_date datetime,
    end_date datetime
  ) 
''')
cur.execute('''
  INSERT INTO project (name, begin_date, end_date)
  VALUES ('taper', '2018-03-01', '2019-01-01')
''')  # step 3b
conn.commit()  # step 4
conn.close()  # step 5

## ORM with SQL Alchemy

ORM: Object-relational mapper. 

Again, the most difficult part is figuring out the connection string. You can get some help from here: 
	* http://docs.sqlalchemy.org/en/latest/core/engines.html
	* For sql server: `r'mssql+pyodbc://SERVER/DATABASE?driver=SQL Server'`
	* Troubleshooting: https://stackoverflow.com/questions/17787042/sqlalchemy-connection-string


In [21]:
import sqlalchemy

In [42]:
from sqlalchemy import create_engine
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime, ForeignKey

Base = declarative_base()

In [43]:
class Project(Base):
    __tablename__ = 'projects'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    begin_date = Column(DateTime)
    end_date = Column(DateTime)
    tasks = relationship('Task')
    
    def __repr__(self):
        return f'<Project "{self.name}": {self.begin_date}-{self.end_date}>'
    
    def __str__(self):
        return f'<Project "{self.name}": {self.begin_date}-{self.end_date}>'

In [44]:
class Task(Base):
    __tablename__ = 'tasks'  # optional, if you want a different name
    
    id = Column(Integer, primary_key=True)
    name = Column(String(20))
    priority = Column(Integer)  # 1=most important
    status_id = Column(Integer, default=0)  # 0=Not started, 1=in progress, 2=done
    project_id = Column(Integer, ForeignKey('projects.id'))
    due_date = Column(DateTime)
    
    def __repr__(self):
        return f'<Task "{self.name}", Due:{self.due_date}, Priority:{self.priority}>'
    
    def __str__(self):
        return f'<Task "{self.name}", Due:{self.due_date}, Priority:{self.priority}>'

In [45]:
connection_str = r'sqlite:///:memory:'  # this for an in-memory only version
eng = create_engine(connection_str)
Base.metadata.create_all(eng)

In [46]:
session = sessionmaker(bind=eng)()

In [47]:
import datetime
p = Project(name='cannabis', 
            begin_date=datetime.datetime(2018, 1, 1), 
            end_date=datetime.datetime(2019, 1, 1))
print(p.id)
session.add(p)
session.commit()
print(p.id)

In [48]:
t1 = Task(name='review timeline',
          priority=2,
          project_id=p.id,
          due_date=datetime.datetime(2018, 5, 1)
          )
print(t1.id)
session.add(t1)
session.commit()
print(t1.id)

None
1


In [50]:
session.query(Task).filter_by(name='review timeline').first()

<Task "review timeline", Due:2018-05-01 00:00:00, Priority:2>

In [51]:
session.query(Task).filter_by(name='review timeline').all()

[<Task "review timeline", Due:2018-05-01 00:00:00, Priority:2>]

In [56]:
# more complex queries
# ilike = case insensitive
session.query(Task).filter(Task.name.like('review%')).order_by(Task.id).all()

[<Task "review timeline", Due:2018-05-01 00:00:00, Priority:2>]

Other options, see http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#common-filter-operators:
* EQUALS: `Task.name == x`
* NOT EQUALS: `Task.name != x`
* IN: `Task.name.in_(x)`
* NOT IN: `~Task.name.in_(x)`
* IS NULL: `Task.name == None`

For `AND`/`OR`:
* `from sqlalchemy import and_, or_`
* `(or_(Task.name == 'x', Task.id != 1))`